In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import os
import shutil
from numpy.linalg import svd
import cv2
import pywt
import matplotlib.pyplot as plt

In [3]:
Normal = {}
Benign = {}
Maligant = {}

In [4]:
FolderList = ['Normal cases', 'Benign cases', 'Malignant cases']
Dict_list = [Normal, Benign, Maligant]

In [5]:
folder_index = 0
for i in range(3):
    print(FolderList[i])
    folder_path = f'C:/vitdocs/lung cancer project/{FolderList[i]}'
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            with Image.open(image_path) as img:
                width, height = img.size
                if f"{width}x{height}" in Dict_list[i].keys():
                    Dict_list[i][f"{width}x{height}"] += 1
                else:
                    if width == 506 and height == 331:
                        print(filename)
                        # break
                    Dict_list[i][f"{width}x{height}"] = 1

Normal cases
Normal case (344).jpg
Benign cases
Malignant cases


In [6]:
Normal

{'512x512': 415, '506x331': 1}

In [7]:
Benign

{'512x512': 120}

In [8]:
Maligant

{'512x512': 501, '511x404': 1, '801x512': 28, '623x512': 31}

In [9]:
NotProper_dimension = []

In [10]:
folder_path = f'C:/vitdocs/lung cancer project/Malignant cases'
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(folder_path, filename)
        with Image.open(image_path) as img:
            print(filename)
            width, height = img.size
            if (width, height) != (512, 512) and (width, height) != (511, 404):
                print(f'Not proper dimensions: {filename}')
                NotProper_dimension.append(filename)


len(NotProper_dimension)

Malignant case (1).jpg
Malignant case (10).jpg
Malignant case (100).jpg
Malignant case (101).jpg
Malignant case (102).jpg
Malignant case (103).jpg
Malignant case (104).jpg
Malignant case (105).jpg
Malignant case (106).jpg
Malignant case (107).jpg
Malignant case (108).jpg
Malignant case (109).jpg
Malignant case (11).jpg
Malignant case (110).jpg
Malignant case (111).jpg
Malignant case (112).jpg
Malignant case (113).jpg
Malignant case (114).jpg
Malignant case (115).jpg
Malignant case (116).jpg
Malignant case (117).jpg
Malignant case (118).jpg
Malignant case (119).jpg
Malignant case (12).jpg
Malignant case (120).jpg
Malignant case (121).jpg
Malignant case (122).jpg
Malignant case (123).jpg
Malignant case (124).jpg
Malignant case (125).jpg
Malignant case (126).jpg
Malignant case (127).jpg
Malignant case (128).jpg
Malignant case (129).jpg
Malignant case (13).jpg
Malignant case (130).jpg
Malignant case (131).jpg
Malignant case (132).jpg
Malignant case (133).jpg
Malignant case (134).jpg
Malign

59

In [11]:
with open('dimension.txt', 'w') as file:
    for filename in NotProper_dimension:
        file.write(f'{filename}\n')

In [12]:
folder_path = 'C:/vitdocs/lung cancer project/Malignant cases'
output_folder = 'C:/vitdocs/lung cancer project/data processed/SVD/Malignant'
os.makedirs(output_folder, exist_ok=True)

In [13]:
def resize_image_svd(image_array, target_size=(512, 512), k=100):
    U, S, Vt = svd(image_array, full_matrices=False)
    U_k = U[:, :k]
    S_k = np.diag(S[:k])
    Vt_k = Vt[:k, :]
    reduced_image = np.dot(U_k, np.dot(S_k, Vt_k))
    reduced_image = np.clip(reduced_image, 0, 255)
    resized_image = Image.fromarray(reduced_image.astype(
        np.uint8)).resize(target_size, Image.LANCZOS)
    return np.array(resized_image)

In [14]:
for filename in NotProper_dimension:
    image_path = os.path.join(folder_path, filename)
    if os.path.isfile(image_path):
        with Image.open(image_path) as img:
            img = img.convert('L')
            img_array = np.array(img)

            resized_img_array = resize_image_svd(
                img_array, target_size=(512, 512), k=100)

            resized_img = Image.fromarray(resized_img_array)
            resized_img.save(os.path.join(output_folder, filename))
            print(f'Resized and saved: {filename}')
    else:
        print(f'File not found: {filename}')

Resized and saved: Malignant case (200).jpg
Resized and saved: Malignant case (201).jpg
Resized and saved: Malignant case (202).jpg
Resized and saved: Malignant case (203).jpg
Resized and saved: Malignant case (204).jpg
Resized and saved: Malignant case (205).jpg
Resized and saved: Malignant case (206).jpg
Resized and saved: Malignant case (207).jpg
Resized and saved: Malignant case (208).jpg
Resized and saved: Malignant case (209).jpg
Resized and saved: Malignant case (210).jpg
Resized and saved: Malignant case (211).jpg
Resized and saved: Malignant case (212).jpg
Resized and saved: Malignant case (213).jpg
Resized and saved: Malignant case (214).jpg
Resized and saved: Malignant case (215).jpg
Resized and saved: Malignant case (216).jpg
Resized and saved: Malignant case (217).jpg
Resized and saved: Malignant case (218).jpg
Resized and saved: Malignant case (219).jpg
Resized and saved: Malignant case (220).jpg
Resized and saved: Malignant case (221).jpg
Resized and saved: Malignant cas

In [15]:
for filename in os.listdir(folder_path):
    if filename in NotProper_dimension:
        continue
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(folder_path, filename)
        with Image.open(image_path) as img:
            print(filename)
            width, height = img.size
            if (width, height) != (511, 404):
                shutil.copy(image_path, os.path.join(output_folder, filename))
                print(f'Copied {filename} to {output_folder}')

Malignant case (1).jpg
Copied Malignant case (1).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (10).jpg
Copied Malignant case (10).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (100).jpg
Copied Malignant case (100).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (101).jpg
Copied Malignant case (101).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (102).jpg
Copied Malignant case (102).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (103).jpg
Copied Malignant case (103).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (104).jpg
Copied Malignant case (104).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (105).jpg
Copied Malignant case (105).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Malignant
Malignant case (106).jpg
Copied Malignant case

In [16]:
Benign

{'512x512': 120}

In [17]:
folder_path = 'C:/vitdocs/lung cancer project/Benign cases/'
output_folder = 'C:/vitdocs/lung cancer project/data processed/SVD/Benign/'
os.makedirs(output_folder, exist_ok=True)

In [18]:
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(folder_path, filename)
        with Image.open(image_path) as img:
            print(filename)
            width, height = img.size
            shutil.copy(image_path, os.path.join(output_folder, filename))
            print(f'Copied {filename} to {output_folder}')

Bengin case (1).jpg
Copied Bengin case (1).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (10).jpg
Copied Bengin case (10).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (100).jpg
Copied Bengin case (100).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (101).jpg
Copied Bengin case (101).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (102).jpg
Copied Bengin case (102).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (103).jpg
Copied Bengin case (103).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (104).jpg
Copied Bengin case (104).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (105).jpg
Copied Bengin case (105).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign/
Bengin case (106).jpg
Copied Bengin case (106).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Benign

In [19]:
folder_path = 'C:/vitdocs/lung cancer project/Normal cases/'
output_folder = 'C:/vitdocs/lung cancer project/data processed/SVD/Normal/'
os.makedirs(output_folder, exist_ok=True)

In [20]:
Normal

{'512x512': 415, '506x331': 1}

In [21]:
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(folder_path, filename)
        with Image.open(image_path) as img:
            print(filename)
            width, height = img.size
            if (width, height) != (506, 331):
                shutil.copy(image_path, os.path.join(output_folder, filename))
                print(f'Copied {filename} to {output_folder}')

Normal case (1).jpg
Copied Normal case (1).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (10).jpg
Copied Normal case (10).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (100).jpg
Copied Normal case (100).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (101).jpg
Copied Normal case (101).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (102).jpg
Copied Normal case (102).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (103).jpg
Copied Normal case (103).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (104).jpg
Copied Normal case (104).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (105).jpg
Copied Normal case (105).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal/
Normal case (106).jpg
Copied Normal case (106).jpg to C:/vitdocs/lung cancer project/data processed/SVD/Normal

In [22]:
def wavelet_transform_and_save(image_path, wavelet_name, save_dir):
    original = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    coeffs2 = pywt.dwt2(original, wavelet_name)
    LL, (LH, HL, HH) = coeffs2
    titles = ['Approximation', 'Horizontal detail',
              'Vertical detail', 'Diagonal detail']
    fig = plt.figure(figsize=(12, 3))
    for i, a in enumerate([LL, LH, HL, HH]):
        ax = fig.add_subplot(1, 4, i + 1)
        ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
        ax.set_title(titles[i], fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    save_path = os.path.join(save_dir, f"{image_name}_{wavelet_name}.png")
    fig.tight_layout()
    plt.savefig(save_path)
    plt.close()

In [23]:
image_folder = "C:/vitdocs/lung cancer project/data processed/SVD/Benign"
save_folder = "C:/vitdocs/lung cancer project/data processed/Pywavelet//Benign"

In [24]:
wavelet_variations = ["bior1.1", "bior1.3", "bior1.5", "db1", "haar", "rbio1.1",  "rbio3.1"]

In [25]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import os
import shutil
from numpy.linalg import svd
import cv2
import pywt
import matplotlib.pyplot as plt


In [26]:
wavelet_variations = ["bior1.1", "bior1.3", "bior1.5",
                      "db1", "haar", "rbio1.1",  "rbio3.1"]

In [27]:
Benign={}


In [28]:
def wavelet_transform_and_save(image_path, wavelet_name, save_dir, dictt):
    # Read the image in grayscale
    original = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Perform 2D wavelet transform
    coeffs2 = pywt.dwt2(original, wavelet_name)
    LL, (LH, HL, HH) = coeffs2  # Approximation, Horizontal, Vertical, Diagonal

    # Function to resize and save the image if not 64x64
    def save_resized_image(image, save_path, dictt):
        image_shape = image.shape
        shape_key = f"{image_shape[0]}x{image_shape[1]}"

        # If the size is not 64x64, resize the image
        if image_shape != (64, 64):
            image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
            shape_key = "64x64"

        # Save the image
        plt.imsave(save_path, image, cmap='gray')

        # Update dictionary with shape count
        if shape_key in dictt:
            dictt[shape_key] += 1
        else:
            dictt[shape_key] = 1

    # Save horizontal (LH) details
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    horizontal_save_path = os.path.join(
        save_dir, f"{image_name}_{wavelet_name}_horizontal.png")
    save_resized_image(LH, horizontal_save_path, dictt)

    # Save vertical (HL) details
    vertical_save_path = os.path.join(
        save_dir, f"{image_name}_{wavelet_name}_vertical.png")
    save_resized_image(HL, vertical_save_path, dictt)

    print(f"Saved horizontal details at: {horizontal_save_path}")
    print(f"Saved vertical details at: {vertical_save_path}")

In [29]:
image_folder = 'C:/vitdocs/lung cancer project/data processed/SVD/Benign/'
save_folder = "C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/"

In [30]:
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [31]:
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)

    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        print(f"Processing {filename}...")

        for wavelet in wavelet_variations:
            wavelet_transform_and_save(image_path, wavelet, save_folder,Benign)

Processing Bengin case (1).jpg...
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_bior1.1_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_bior1.1_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_bior1.3_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_bior1.3_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_bior1.5_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_bior1.5_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/Bengin case (1)_db1_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data pr

In [32]:
Benign

{'64x64': 1680}

In [33]:
Malignant = {}

In [34]:
image_folder = 'C:/vitdocs/lung cancer project/data processed/SVD/Malignant/'
save_folder = "C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/"

In [35]:
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [36]:
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)

    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        print(f"Processing {filename}...")

        for wavelet in wavelet_variations:
            wavelet_transform_and_save(image_path, wavelet, save_folder,Malignant)

Processing Malignant case (1).jpg...
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_bior1.1_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_bior1.1_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_bior1.3_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_bior1.3_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_bior1.5_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_bior1.5_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/Malignant case (1)_db1_horizontal.png
Saved vertical detai

In [37]:
Malignant

{'64x64': 7840}

In [38]:
Normal ={}

In [39]:
image_folder = 'C:/vitdocs/lung cancer project/data processed/SVD/Normal/'
save_folder = "C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/"

In [40]:
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [41]:
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)

    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        print(f"Processing {filename}...")

        for wavelet in wavelet_variations:
            wavelet_transform_and_save(image_path, wavelet, save_folder,Normal)

Processing Normal case (1).jpg...
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_bior1.1_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_bior1.1_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_bior1.3_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_bior1.3_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_bior1.5_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_bior1.5_vertical.png
Saved horizontal details at: C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/Normal case (1)_db1_horizontal.png
Saved vertical details at: C:/vitdocs/lung cancer project/data pr

In [42]:
Normal

{'64x64': 5810}

In [43]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define folders and titles
categories = ['Benign', 'Malignant', 'Normal']
base_input_folder = 'C:/vitdocs/lung cancer project/data processed/Pywavelet/'
base_output_folder = 'C:/vitdocs/lung cancer project/data processed/filters/'

# Create output folders if they don't exist
if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)


In [44]:

# Function to apply filters and save the image grid
def apply_filters_and_save(image_path, output_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply various filters
    gaussian_filtered = cv2.GaussianBlur(image, (5, 5), 0)
    median_filtered = cv2.medianBlur(image, 5)
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    sobelx = cv2.convertScaleAbs(sobelx)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    sobely = cv2.convertScaleAbs(sobely)
    laplacian_filtered = cv2.Laplacian(image, cv2.CV_64F)
    laplacian_filtered = cv2.convertScaleAbs(laplacian_filtered)
    canny_edges = cv2.Canny(image, 100, 200)
    blurred = cv2.GaussianBlur(image, (9, 9), 10.0)
    unsharp_mask = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    gabor_filter = cv2.getGaborKernel((21, 21), 5, np.pi/4, 10, 0.5, 0, ktype=cv2.CV_32F)
    gabor_filtered = cv2.filter2D(image, cv2.CV_32F, gabor_filter)
    gabor_filtered = cv2.convertScaleAbs(gabor_filtered)
    bilateral_filtered = cv2.bilateralFilter(image, 9, 75, 75)
    denoised_image = cv2.fastNlMeansDenoising(image, None, 30, 7, 21)
    kernel = np.ones((5, 5), np.uint8)
    morph_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

    # List of all filtered images including the original
    images = [image, gaussian_filtered, median_filtered, sobelx, sobely, laplacian_filtered,
              canny_edges, unsharp_mask, gabor_filtered, bilateral_filtered,
              denoised_image, morph_image]

    # Titles for the grid layout, including the original image
    titles = ['Original', 'Gaussian', 'Median', 'Sobel X', 'Sobel Y', 'Laplacian',
              'Canny', 'Unsharp', 'Gabor', 'Bilateral', 'Denoised', 'Morphology']

    # Set up a grid layout (e.g., 4x3 grid)
    fig, axes = plt.subplots(4, 3, figsize=(15, 15))

    # Plot each image in the grid
    for ax, img, title in zip(axes.flat, images, titles):
        ax.imshow(img, cmap='gray')
        ax.set_title(title)
        ax.axis('off')

    # Adjust layout and save the grid of images
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# Process each category
for category in categories:
    input_folder = os.path.join(base_input_folder, category)
    output_folder = os.path.join(base_output_folder, category)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]

    for i, image_file in enumerate(image_files[:10]):
        image_path = os.path.join(input_folder, image_file)
        output_image_path = os.path.join(output_folder, f'filtered_grid_image_{i+1}.png')

        # Apply filters and save the result
        apply_filters_and_save(image_path, output_image_path)

    print(f"Processed and saved {len(image_files[:10])} images in '{output_folder}'")


Processed and saved 10 images in 'C:/vitdocs/lung cancer project/data processed/filters/Benign'
Processed and saved 10 images in 'C:/vitdocs/lung cancer project/data processed/filters/Malignant'
Processed and saved 10 images in 'C:/vitdocs/lung cancer project/data processed/filters/Normal'


In [45]:
import os
import cv2
import matplotlib.pyplot as plt

# Combined folders for different cases
input_folders = {
    'Benign': 'C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/',
    'Malignant': 'C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/',
    'Normal': 'C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/'
}

# Set output folder to "final filter"
base_output_folder = 'C:/vitdocs/lung cancer project/data processed/final filter/'

if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)

In [46]:
Normal

{'64x64': 5810}

In [47]:
Benign

{'64x64': 1680}

In [48]:
Malignant

{'64x64': 7840}

In [51]:
# Function to apply Gaussian blur, resize, and save the result for 'Normal' case
def apply_gaussian_blur_and_save(image_path, output_path, output_size=(64, 64)):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian blur filter
    gaussian_filtered = cv2.GaussianBlur(image, (5, 5), 0)

    # Resize to 64x64
    resized_img = cv2.resize(gaussian_filtered, output_size)

    # Save the resized and blurred image
    plt.imshow(resized_img, cmap='gray')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Define the folder paths for the 'Normal' case
normal_input_folder = 'C:/vitdocs/lung cancer project/data processed/Pywavelet/Benign/'
normal_output_folder = 'C:/vitdocs/lung cancer project/data processed/final filter/Benign'

# Make sure the output folder exists
if not os.path.exists(normal_output_folder):
    os.makedirs(normal_output_folder)

# List the images in the input folder
image_files = [f for f in os.listdir(normal_input_folder) if f.endswith(('.jpg', '.png'))]

# Process each image in the 'Normal' case
for i, image_file in enumerate(image_files):
    image_path = os.path.join(normal_input_folder, image_file)
    output_image_path = os.path.join(normal_output_folder, f'gaussian_blur_image_{i+1}.png')

    # Apply Gaussian blur, resize to 64x64, and save the result
    apply_gaussian_blur_and_save(image_path, output_image_path, output_size=(64, 64))

print(f"Processed and saved {len(image_files)} images in '{normal_output_folder}'")


Processed and saved 1680 images in 'C:/vitdocs/lung cancer project/data processed/final filter/Benign'


In [53]:
# Define the folder paths for the 'Normal' case
normal_input_folder = 'C:/vitdocs/lung cancer project/data processed/Pywavelet/Normal/'
normal_output_folder = 'C:/vitdocs/lung cancer project/data processed/final filter/Normal'

# Make sure the output folder exists
if not os.path.exists(normal_output_folder):
    os.makedirs(normal_output_folder)

# List the images in the input folder
image_files = [f for f in os.listdir(normal_input_folder) if f.endswith(('.jpg', '.png'))]

# Process each image in the 'Normal' case
for i, image_file in enumerate(image_files):
    image_path = os.path.join(normal_input_folder, image_file)
    output_image_path = os.path.join(normal_output_folder, f'gaussian_blur_image_{i+1}.png')

    # Apply Gaussian blur, resize to 64x64, and save the result
    apply_gaussian_blur_and_save(image_path, output_image_path, output_size=(64, 64))

print(f"Processed and saved {len(image_files)} images in '{normal_output_folder}'")


Processed and saved 5810 images in 'C:/vitdocs/lung cancer project/data processed/final filter/Normal'


In [54]:
# Define the folder paths for the 'Normal' case
normal_input_folder = 'C:/vitdocs/lung cancer project/data processed/Pywavelet/Malignant/'
normal_output_folder = 'C:/vitdocs/lung cancer project/data processed/final filter/Malignant'

# Make sure the output folder exists
if not os.path.exists(normal_output_folder):
    os.makedirs(normal_output_folder)

# List the images in the input folder
image_files = [f for f in os.listdir(normal_input_folder) if f.endswith(('.jpg', '.png'))]

# Process each image in the 'Normal' case
for i, image_file in enumerate(image_files):
    image_path = os.path.join(normal_input_folder, image_file)
    output_image_path = os.path.join(normal_output_folder, f'gaussian_blur_image_{i+1}.png')

    # Apply Gaussian blur, resize to 64x64, and save the result
    apply_gaussian_blur_and_save(image_path, output_image_path, output_size=(64, 64))

print(f"Processed and saved {len(image_files)} images in '{normal_output_folder}'")


Processed and saved 7840 images in 'C:/vitdocs/lung cancer project/data processed/final filter/Malignant'


In [59]:
import os
from PIL import Image
import numpy as np
from scipy.linalg import svd

def resize_image_svd(image_array, target_size=(64, 64), k=100):
    U, S, Vt = svd(image_array, full_matrices=False)
    U_k = U[:, :k]
    S_k = np.diag(S[:k])
    Vt_k = Vt[:k, :]
    reduced_image = np.dot(U_k, np.dot(S_k, Vt_k))
    reduced_image = np.clip(reduced_image, 0, 255)
    resized_image = Image.fromarray(reduced_image.astype(np.uint8)).resize(target_size, Image.LANCZOS)
    return np.array(resized_image)

# Define paths for normal, benign, and malignant cases
folders = {
    'normal': 'C:/vitdocs/lung cancer project/data processed/final filter/Normal',
    'benign': 'C:/vitdocs/lung cancer project/data processed/final filter/Benign',
    'malignant': 'C:/vitdocs/lung cancer project/data processed/final filter/Malignant'
}

for case, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        if os.path.isfile(image_path):
            with Image.open(image_path) as img:
                img = img.convert('L')  # Convert to grayscale
                img_array = np.array(img)

                # Resize to 64x64 using SVD
                resized_img_array = resize_image_svd(img_array, target_size=(64, 64), k=100)

                # Save the resized image back to the same folder, overwriting the original file
                resized_img = Image.fromarray(resized_img_array)
                resized_img.save(image_path)
                print(f'Resized and saved: {filename} in {case} folder')
        else:
            print(f'File not found: {filename} in {case} folder')


Resized and saved: gaussian_blur_image_1.png in normal folder
Resized and saved: gaussian_blur_image_10.png in normal folder
Resized and saved: gaussian_blur_image_1064.png in normal folder
Resized and saved: gaussian_blur_image_1065.png in normal folder
Resized and saved: gaussian_blur_image_1066.png in normal folder
Resized and saved: gaussian_blur_image_1067.png in normal folder
Resized and saved: gaussian_blur_image_1068.png in normal folder
Resized and saved: gaussian_blur_image_1069.png in normal folder
Resized and saved: gaussian_blur_image_1070.png in normal folder
Resized and saved: gaussian_blur_image_1071.png in normal folder
Resized and saved: gaussian_blur_image_1072.png in normal folder
Resized and saved: gaussian_blur_image_1073.png in normal folder
Resized and saved: gaussian_blur_image_1074.png in normal folder
Resized and saved: gaussian_blur_image_1075.png in normal folder
Resized and saved: gaussian_blur_image_1076.png in normal folder
Resized and saved: gaussian_bl